In [1]:
!pip install --upgrade -q gspread
!pip install -U -q PyDrive


import gspread
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import drive as gdrive

gdrive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
y_test_fname = '/content/gdrive/MyDrive/ML práctico 2022/2022-2C/y_test.jl'

In [5]:
with open(y_test_fname) as f:
  y_test = list(map(float, f.read().split('\n')))

In [69]:
from collections import Counter
from time import sleep
from sklearn.metrics import mean_squared_error, mean_absolute_error, \
      mean_absolute_percentage_error
from IPython.display import clear_output, display

class Evaluator:
  def __init__(self):
    self._cached_scores = {}

  def get_submissions(self):
    worksheet = gc.open('Taller de model selection (Responses)')
    rows = worksheet.worksheets()[0].get_all_values()
    header = rows[0]
    rows = [dict(zip(header, row)) for row in rows[1:]]
    return rows
    
  def get_competition_table(self):
    rows = self.get_submissions()

    team_submissions = Counter()
    team_best = {}

    for row in rows:
      team_name = row['Nombre equipo']
      team_submissions[team_name] += 1
      
      id = row['Submission'].split('=')[-1]
      result = self.score_solution(id)

      result.update(row)

      if team_name not in team_best:
        team_best[team_name] = result
      elif result['mse'] < team_best[team_name]['mse']:
        team_best[team_name] = result

    docs = []
    for team_name, best_result in team_best.items():
      doc = {}
      doc.update(best_result)
      doc['team_name'] = team_name
      doc['submissions'] = team_submissions[team_name]
      docs.append(doc)

    df = pd.DataFrame(docs).sort_values('mse')
    now = datetime.now()
    secs_ago = lambda t: pd.Timedelta(now - datetime.strptime(t, '%m/%d/%Y %H:%M:%S')).round('S')
    df['Secs ago'] = df.Timestamp.apply(secs_ago)
    return df[['team_name', 'Secs ago', 'submissions', 
             'mse', 'mae', 'mape', 'Comment']]

  def score_solution(self, id):
      if id not in self._cached_scores:
        downloaded = drive.CreateFile({'id':id})   # replace the id with id of file you want to access
        downloaded.GetContentFile('tmp_submission.csv') 
        with open('tmp_submission.csv') as f:
          y_pred = list(map(float, f.read().split()))
          
        self._cached_scores[id] = dict(
            mse = mean_squared_error(y_test, y_pred),
            mae = mean_absolute_error(y_test, y_pred),
            mape = mean_absolute_percentage_error(y_test, y_pred),
        )

      return self._cached_scores[id]


In [70]:
ev = Evaluator()

In [ ]:
from datetime import datetime
from collections import Counter
from time import sleep
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output, display, display_markdown

while True:
  prev_df = df
  df = ev.get_competition_table()
  if len(df) != len(prev_df) or (df.mse == prev_df.mse).all(): 
    clear_output()
    display_markdown(f'**Hora**: {datetime.now().strftime("%H:%M:%S")}', raw=True)
    display(df)
  sleep(10)

**Hora**: 20:41:09

,team_name,Secs ago,submissions,mse,mae,mape,Comment
1,LGBM title + numeric + l2 + l3,0 days 03:15:30,1,6.839015e+10,94342.019152,0.637745,"mismas features que el anterior, con lgbm"
0,LR title + numeric + l2 + l3,0 days 03:27:56,1,1.146365e+11,151918.279454,1.035083,Regresion logistica con title fasttext + numer...
